In [165]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import roc_auc_score
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.preprocessing import StandardScaler

In [177]:
trainDF = pd.read_csv("Train.csv")
testDF = pd.read_csv("Test.csv")

In [178]:
trainDF.date = pd.to_datetime(trainDF.date)
years = pd.DatetimeIndex(trainDF.date).year
months = pd.DatetimeIndex(trainDF.date).month
trainDF['year'] = years
trainDF['month'] = months
trainDF = trainDF.drop(['date'], axis=1)
testDF['year'] = pd.DatetimeIndex(testDF.date).year
testDF['months'] = pd.DatetimeIndex(testDF.date).month
testDF = testDF.drop(['date'], axis=1)

In [179]:
trainDF = trainDF.drop(['g_lift'], axis=1)
trainDF = trainDF.drop(['kw9'], axis=1)
trainDF = trainDF.drop(['kw8'], axis=1)
trainDF = trainDF.fillna(trainDF.mean())
testDF = testDF.drop(['g_lift'], axis=1)
testDF = testDF.drop(['kw9'], axis=1)
testDF = testDF.drop(['kw8'], axis=1)
testDF = testDF.fillna(testDF.mean())
len(trainDF)

100000

In [180]:
# fig, ax = plt.subplots(figsize=(15,15))
# corrM = trainDF.corr()
# sns.heatmap(corrM)

In [181]:
YTrain = np.array(trainDF.price)
XTrain = np.array(trainDF.drop(['price'], axis=1))
#XTest = np.array(testDF)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(XTrain, YTrain, test_size=0.33, random_state=10)
X_train = np.array(Xtrain)
X_test = np.array(Xtest)
y_train = np.array(Ytrain)
y_test = np.array(Ytest)

In [189]:
nb = LGBMRegressor( n_estimators=500, learning_rate=0.1)
hr = HistGradientBoostingRegressor()
xg = xgb.XGBRegressor(n_estimators=500)
nb.fit(X_train, y_train)
hr.fit(X_train, y_train)
xg.fit(X_train, y_train)
solutions = pd.DataFrame()
solutions['nbres'] = nb.predict(X_test)
solutions ['hrres'] = hr.predict(X_test)
solutions ['xgres'] = xg.predict(X_test)
solutions_sk = StandardScaler().fit_transform(solutions)
solutions.head()

,nbres,hrres,xgres
0,5.185461e+06,5.873754e+06,4144268.0
1,3.521494e+06,3.063709e+06,4257014.0
2,2.925359e+06,3.091375e+06,2505673.5
3,3.015761e+06,3.815111e+06,3324873.0
4,6.070923e+06,5.925818e+06,7415804.5


In [190]:
lr = Lasso()
lr.fit(solutions_sk, y_test)

Lasso()

In [191]:
mean_absolute_error(lr.predict(solutions_sk), y_test)

850223.5790438203

In [206]:
xgbR = xgb.XGBRegressor(n_estimators=500)
xgbR.fit(X_train, y_train)
mean_absolute_error(xgbR.predict(X_train), y_train)

513751.1623697382

In [207]:
testDF = testDF.drop(['price'], axis=1)
# sl = pd.DataFrame()
# sl ['nbres'] = nb.predict(testDF)
# sl ['nrres'] = hr.predict(testDF)
# sl ['xgres'] = xg.predict(testDF)
testDF["price"] = xgbR.predict(testDF)
testDF[["id", "price"]].to_csv("dz5_xgb.csv", index=False)